In [1]:
import requests
import json
import os
import pandas as pd
import re
import time

In [2]:
def get_url(query, page, limit = 10):
    # sort by date inserted
    #url = 'https://forum.alfemminile.com/api/v2/search?domain=discussions&query={query}&sort=-dateInserted&page={page}&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit={limit}&locale=it'.format(query = query, page = page, limit = limit)

    # default sort : relevance
    #url = 'https://forum.alfemminile.com/api/v2/search?domain=discussions&query={query}&page={page}&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit={limit}&locale=it'.format(query = query, page = page, limit = limit)

    # sort by date inserted and filter by date
    url = 'https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&query={query}&page={page}&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit={limit}&locale=it'.format(query = query, page = page, limit = limit)
    
    return url
    

In [3]:
queries = [
    'depressione',
    'ansia',
    'solitudine',
    'suicidio'
]
max_pages = 2

In [4]:
def clean_text(df):
    df['bodyPlainText'] = df['bodyPlainText'].str.replace("\n", " ")
    df['bodyPlainText'] = df['bodyPlainText'].str.replace("\r", " ")
    df['bodyPlainText'] = df.apply(lambda x: x['bodyPlainText'].removeprefix(x['name']), axis=1)
    df['bodyPlainText'] = df['bodyPlainText'].str.strip()

    return df

In [7]:
# query per categoria
df = pd.DataFrame(columns = ['query', 'recordID','insertUserID','name','bodyPlainText','dateInserted'])
errors = []
for query in queries:
    for page in range(max_pages):       
        url = get_url(query, page + 1)
        print(url)
        response = requests.request("GET", url)
        time.sleep(2)
        if response.status_code == 200:
            posts = json.loads(response.text)
            if len(posts) > 0:
                df_page = pd.DataFrame(posts)                
                df_page['query'] = query
                df_page = clean_text(df_page)                
                df = pd.concat([df, df_page[df.columns.tolist()]])                
            else:
                break
            df.to_csv('psicologia-dati/forum.csv', index = False)
        else:
            errors.append(url)



https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&query=depressione&page=1&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it
https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&query=depressione&page=2&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it
https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&query=ansia&page=1&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it
https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&query=ansia&page=2&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it
https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort

In [8]:
# forum di psicologia
n_pages = 2
file_name = 'psicologia-dati/psicologia.csv'
columns = ['query', 'recordID','insertUserID','name','bodyPlainText','dateInserted']
errors = []

for page in range(n_pages):       
    #url = 'https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2004-01-01,2024-12-31]&sort=-dateInserted&page={page}&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit={limit}&locale=it'.format(page = page + 1, limit = 10)
    url = 'https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2018-01-01,2018-12-31]&sort=-dateInserted&page={page}&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit={limit}&locale=it'.format(page = page + 1, limit = 10)
    print(url)
    response = requests.request("GET", url)
    time.sleep(2)
    if response.status_code == 200:
        posts = json.loads(response.text)
        if len(posts) > 0:
            df_page = pd.DataFrame(posts)                
            df_page['query'] = 'psicologia'
            df_page = df_page[columns]
            df_page = clean_text(df_page)              
            df_page.to_csv(file_name, mode = 'a', index = False, header = not os.path.exists(file_name))              
        else:
            break
        
    else:
        errors.append(url)

https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2018-01-01,2018-12-31]&sort=-dateInserted&page=1&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it
https://forum.alfemminile.com/api/v2/search?domain=discussions&dateInserted=[2018-01-01,2018-12-31]&sort=-dateInserted&page=2&types[0]=discussion&recordTypes[0]=discussion&categoryIDs[0]=11&limit=10&locale=it


In [9]:
len(df)

77

In [10]:
df.head()

,query,recordID,insertUserID,name,bodyPlainText,dateInserted
0,depressione,6101660,34992711,Lei assume dei farmaci,Lei 30 anni. Da poco di un anno conviviamo ins...,2024-12-30T15:11:32+00:00
1,depressione,6100943,34996060,"Terapia del sogno, è valida per tutte le forme...",Quante volte abbiamo sentito di gesti estremi ...,2024-12-14T16:46:27+00:00
2,depressione,6100658,34994356,Tristezza nel periodo natalizio,"Ogni anno ci ricasco, la mia ansia e la mia de...",2024-12-09T17:31:03+00:00
3,depressione,6100590,34969891,Come aiutare mia nonna,"Buongiorno, scrivo questo post perché sono ann...",2024-12-08T16:58:06+00:00
4,depressione,6100201,34995593,Mobbing post maternità,"Buonasera a tutt*, cerco un consiglio riguardo...",2024-12-01T18:19:23+00:00


In [62]:
import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

def save_sql_table(df, table):
    connection_url = URL.create(
        "mssql+pyodbc",
        host=".",
        port=1433,
        database="NLP",
        query={
            "driver": "ODBC Driver 18 for SQL Server",
            "Encrypt": "yes",
            "TrustServerCertificate": "yes",
            "TrustedConnection": "yes"
        },
    )
    engine = create_engine(connection_url)
    conn = engine.connect()

    df.to_sql(table, con=engine, if_exists='append', index=False, chunksize=1000)

    conn.close()

In [58]:
df_depression = df[df['query'] != 'ansia'][['recordID','name','bodyPlainText']].drop_duplicates()

In [63]:
save_sql_table(df_depression, 'DEPRESSION_MESSAGES')